In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.linalg as la
import networkx as nx

In [2]:
edges = []
with open('./soc-wiki-vote.txt') as f:
    for line in f:
        one_row = line.split()
        (u, v) = int(one_row[0]), int(one_row[1])
        edges.append((u,v))
    
edges[:5]

[(8, 1), (19, 1), (36, 1), (3, 2), (4, 2)]

In [3]:
G = nx.DiGraph()
G.add_edges_from(edges)

In [14]:
def power_iter(
    G,
    beta=0.8,
    max_iter=100,
    tol=1.0e-7,
    init_vector=None,
    dangling=None):
    
    M = nx.to_numpy_array(G).T # transpose to convert to column stochastic matrix
    N = M.shape[0]
    
    if N == 0:
        return {}
    
    if init_vector is None:
        x = np.repeat(1.0 / N, N)
    else:
        x = init_vector
        
    if dangling is None:
        dangling_weights = np.repeat(1.0, N)
    else:
        dangling_weights = np.repeat(dangling, N)
    
    # if d_i = 0, preprocess matrix M to remove all dead ends. 
    # follow random teleport links with probability 1.0 from dead-ends or dangling_weights
    
    dangling_nodes = np.where(np.sum(M, axis=0) == 0)[0]

    for node in dangling_nodes:
        M[:, node] = dangling_weights

    
    M = M / np.sum(M, axis=0, keepdims=1)
    r = np.repeat(1.0 / N, N)
    p = np.repeat(1.0 / N, N)
    
    for it in range(max_iter):
        rlast = r
        r = beta * (M @ r) + (1 - beta) * p
        mean_abs_error = np.absolute(r - rlast).sum() / N
        print(it, mean_abs_error)
        if mean_abs_error < tol:
            std = np.round(np.std(r), 3)
            print(f'Converged at it= {it}, the standard deviation of r^(t)= {std}')
            
            nodes = list(G.nodes)
            ranks = {}
            for i, j in zip(nodes, range(r.shape[0])):
                ranks[i] = r[j]
                
            return ranks


In [5]:
l = np.arange(start=0.1, stop=1.0, step=0.1)

for b in l:
    print(f'β= {round(b, 3)}, ', end='')
    power_iter(G, beta=b)
#     print()

β= 0.1, Converged at it= 3, the standard deviation of r^(t)= 0.0
β= 0.2, Converged at it= 4, the standard deviation of r^(t)= 0.0
β= 0.3, Converged at it= 5, the standard deviation of r^(t)= 0.001
β= 0.4, Converged at it= 6, the standard deviation of r^(t)= 0.001
β= 0.5, Converged at it= 7, the standard deviation of r^(t)= 0.001
β= 0.6, Converged at it= 8, the standard deviation of r^(t)= 0.001
β= 0.7, Converged at it= 10, the standard deviation of r^(t)= 0.001
β= 0.8, Converged at it= 12, the standard deviation of r^(t)= 0.002
β= 0.9, Converged at it= 15, the standard deviation of r^(t)= 0.002


In [15]:
mypagerank = power_iter(G, beta=0.8)

0 0.0008322018191508486
1 0.00028969370230235044
2 0.00011159276536412762
3 4.994995905292891e-05
4 2.0759183297629454e-05
5 9.974442622604338e-06
6 4.849598069265756e-06
7 2.446212574859245e-06
8 1.2286063933486546e-06
9 6.055855748621117e-07
10 3.0309764344351776e-07
11 1.5655784828279026e-07
12 8.585693065163856e-08
Converged at it= 12, the standard deviation of r^(t)= 0.002


In [9]:
mypagerank[1]

0.002687548854408918

In [11]:
mypagerank = power_iter(G, beta=0.8)
nxpagerank = nx.pagerank(G, alpha=0.8, tol=1e-7)

s = 0
for i in list(mypagerank):
    if round(mypagerank[i], 6) != round(nxpagerank[i], 6):
        s += 1
        
print(f'number of diferences: {s}')

Converged at it= 12, the standard deviation of r^(t)= 0.002
number of diferences: 0
